In [1]:
#REGLER MIT HISTOGRAMM SCHWEIZ UND NACHBARLÄNDER ABER LAND ------------------------------------------------------------------------
#Import
import pandas as pd
import plotly.express as px
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
#neue Imports
import math
import dash_bootstrap_components as dbc

#mit angepasster Datensatz
#Dateipfad
url = "https://raw.githubusercontent.com/hon3ypi/DashboardDesignFS23/main/newDataset.csv"

#Daten aus CSV-Datei laden
datafile = pd.read_csv(url)

#SPIELBEREICH AB HIER ------------------------------------------------------------------------
#Array mit Schweiz und Nachbarländer
countries = ['Switzerland', 'Germany', 'France', 'Austria', 'Italy', 'Liechtenstein']

#Daten für die ausgewählten Länder und Jahre filtern
filtComb = datafile.query('GeoAreaName in @countries')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 28, saw 367


In [ ]:
#ANZEIGENZEUGS AB HIER ------------------------------------------------------------------------
#Web und Serverzeugs
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#Diagramm
#Liniendiagramm
scatfig = px.line(filtComb, x="Year", y="Value", color='GeoAreaName')
#Histogramm
hisfig = px.histogram(filtComb, x="Year", y="Value", color='GeoAreaName', barmode='group')

#Layout
app.layout = html.Div([
    html.H1("Proportion of women in managerial positions", style={'text-align': 'center'}),
    html.H4("Numbers and facts about women in managerial positions in the world", style={'text-align': 'center'}),
    
    #HISTOGRAMM
    html.Label("Filter year (Aufgabe 5a):"),
    dcc.RangeSlider(
        id='scatter-slider',
        min=datafile['Year'].min(),
        max=datafile['Year'].max(),
        value=[datafile['Year'].min(), datafile['Year'].max()],
        marks={str(value): str(value) for value in datafile['Year'].unique()},
        step=None
    ),
    dcc.Graph(id='hist-plot', figure=hisfig),
    
    #STREU, NEU LINIEN (bennenung noch scat)
    html.Label("Select country (Aufgabe 5b):"),
    dcc.Dropdown(
        id='line-country-dropdown',
        options=[{'label': color, 'value': color} for color in filtComb['GeoAreaName'].unique()],
        value='Switzerland'
    ),
    dcc.Graph(id='line-plot')
])


#Linien diagramm
@app.callback(Output('line-plot', 'figure'),
              Input('line-country-dropdown', 'value'),
              Input('scatter-slider', 'value'))
def update_line_plot(color, value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.line(filtered_data, x="Year", y="Value", color='GeoAreaName')
    
    # Farben beibehalten und nur das ausgewählte Land anzeigen
    for i, data in enumerate(fig.data):
        if data.name == color:
            fig.data[i].visible = True
        else:
            fig.data[i].visible = False
    
    return fig

#Histogramm
@app.callback(Output('hist-plot', 'figure'),
              Input('scatter-slider', 'value'))
def update_histogram_plot(value):
    filtered_data = filtComb[(filtComb['Year'] >= value[0]) & (filtComb['Year'] <= value[1])]
    fig = px.histogram(filtered_data, x="Year", y="Value", color='GeoAreaName', barmode='group')
    return fig


if __name__ == '__main__':
    app.run_server(debug=False)
